Instala a biblioteca
```
pip install deltalake 
```

In [3]:
import pandas as pd
from deltalake import DeltaTable
from deltalake.writer import write_deltalake
import deltalake

In [4]:
# le um arquivo csv e converte para dataframe
df = pd.read_csv("Data7602DescendingYearOrder.csv")

In [5]:
df.head()

,anzsic06,Area,year,geo_count,ec_count
0,A,A100100,2022,93,190
1,A,A100200,2022,138,190
2,A,A100300,2022,6,25
3,A,A100400,2022,57,50
4,A,A100500,2022,57,95


In [6]:
df.shape

(5985364, 5)

Transforma o dataframe em um **data lakehouse** !!!

In [7]:
write_deltalake(
    table_or_uri = 'delta_table', # URI of a table or a DeltaTable object.
    data = df,                    # Data to write
    mode = 'overwrite',
    description = 'dataframe created by a pandas dataframe',
    partition_by = ['year'])      # List of columns to partition the table by.